In [1]:
import numpy as np
import mysql.connector

# Basic MySQL stuff

This notebook follows the minicourse video series from https://www.youtube.com/playlist?list=PLzMcBGfZo4-l5kVSNVKGO60V6RkXAVtp-

## First: How to access the database and create new tables

In [4]:
### CONNECT TO SERVER AND CREATE DATABASE

# connect to server
db = mysql.connector.connect(host="localhost",
                            user = "root", passwd = "root")

# create a cursor to do queries
mycursor = db.cursor()
# create testdatabase
mycursor.execute("CREATE DATABASE testdatabase")

###########

In [2]:
# Now access the database
# connect to server
db = mysql.connector.connect(host="localhost",
                            user = "root", passwd = "root", database='testdatabase')
mycursor = db.cursor()

Lets create a new table caled person with columms  (col1 variable type, col2 variabtyp2, ...)

#### We'll use the commands below, but for more options see https://www.w3schools.com/sql/sql_syntax.asp

VARCHAR(n): string of n digits,  smallint: allocate small memory,  UNSIGNED: just positive (no sign),  PRIMARY KEY: unique ID,  AUTO_INCREMENT: modify id for every new entry
}

In [19]:
mycursor.execute("CREATE TABLE person (name VARCHAR(50), age smallint UNSIGNED, personID int PRIMARY KEY AUTO_INCREMENT)")

In [20]:
# check the description of table
mycursor.execute("DESCRIBE Person")

for x in mycursor:
    print(x)

('name', 'varchar(50)', 'YES', '', None, '')
('age', 'smallint unsigned', 'YES', '', None, '')
('personID', 'int', 'NO', 'PRI', None, 'auto_increment')


In [21]:
# add line
mycursor.execute("INSERT INTO Person (name, age) VALUES ('Doug', 26)")

In [22]:
# But its better to do it like this:
new = ('Douglas', 26)
mycursor.execute("INSERT INTO Person (name, age) VALUES (%s, %s)", (new[0], new[1]))

# commit changes to database
db.commit()

In [23]:
# in order to read the table we do
mycursor.execute("SELECT * FROM person")

for x in mycursor:
    print(x)

('Doug', 26, 1)
('Douglas', 26, 2)


In [24]:
# if we want to delete a table we use
mycursor.execute("Drop TABLE Person")

Lets do a new one

In [46]:
from datetime import datetime

#mycursor.execute("Drop TABLE test")

# Enum(...) sets this variable to one of other options
mycursor.execute("CREATE TABLE test (name VARCHAR(50) NOT NULL, created datetime NOT NULL, gender ENUM('M', 'F', 'O') NOT NULL, id int PRIMARY KEY NOT NULL AUTO_INCREMENT)")

mycursor.execute("INSERT INTO test (name, created, gender) VALUES (%s, %s,%s) ", ("Doug", datetime.now(), "M"))
mycursor.execute("INSERT INTO test (name, created, gender) VALUES (%s, %s,%s) ", ("Dalila", datetime.now(), "F"))
mycursor.execute("INSERT INTO test (name, created, gender) VALUES (%s, %s,%s) ", ("bob", datetime.now(), "M"))
mycursor.execute("INSERT INTO test (name, created, gender) VALUES (%s, %s,%s) ", ("marley", datetime.now(), "M"))

db.commit()


In [47]:
# Lets show all the entries with the "M" gender, and order by a column of our choosing
mycursor.execute("SELECT * FROM test WHERE gender = 'M' ORDER BY id DESC")
for x in mycursor:
    print(x)
    
print('\n')
# Or we can select only one property from that giver table entry, like the name
mycursor.execute("SELECT name FROM test WHERE gender = 'M' ORDER BY id DESC")
for x in mycursor:
    print(x[0])

('marley', datetime.datetime(2022, 12, 8, 17, 32, 41), 'M', 4)
('bob', datetime.datetime(2022, 12, 8, 17, 32, 41), 'M', 3)
('Doug', datetime.datetime(2022, 12, 8, 17, 32, 41), 'M', 1)


marley
bob
Doug


In [48]:
# lets try adding a new collumn to save that person's favorite food
mycursor.execute("ALTER TABLE test ADD COLUMN food VARCHAR(20) NOT NULL")

# look at table
mycursor.execute("DESCRIBE test")
for x in mycursor:
    print(x)

('name', 'varchar(50)', 'NO', '', None, '')
('created', 'datetime', 'NO', '', None, '')
('gender', "enum('M','F','O')", 'NO', '', None, '')
('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('food', 'varchar(20)', 'NO', '', None, '')


In [49]:
# if you want to delete a collumn use DROP
mycursor.execute("ALTER TABLE test DROP food")

# look at table
mycursor.execute("DESCRIBE test")
for x in mycursor:
    print(x)

('name', 'varchar(50)', 'NO', '', None, '')
('created', 'datetime', 'NO', '', None, '')
('gender', "enum('M','F','O')", 'NO', '', None, '')
('id', 'int', 'NO', 'PRI', None, 'auto_increment')


In [50]:
# if we want to change a column
mycursor.execute("ALTER TABLE test CHANGE name first_name varchar(20) NOT NULL")

# NOTE: when changing a non-empity table, make sure the new variable type and size
#       are compatible with the previously set values

# look at table
mycursor.execute("DESCRIBE test")
for x in mycursor:
    print(x)
    
mycursor.execute("Drop TABLE test")

('first_name', 'varchar(20)', 'NO', '', None, '')
('created', 'datetime', 'NO', '', None, '')
('gender', "enum('M','F','O')", 'NO', '', None, '')
('id', 'int', 'NO', 'PRI', None, 'auto_increment')


## Foreign Keys & Relating Tables 

Lets see some examples of how to link and relate different tables

In [74]:
# connect to server
db = mysql.connector.connect(host="localhost",
                            user = "root", passwd = "root", database="testdatabase")


users = [('Doug', 'medoug7', 'senha123'),
         ('Emilia', 'emil', 'senhaaa'),
         ('Fred', 'fredfredfred', 'senha1234'),
         ('Otavia', 'otah', 'senhaaa')]

scores = [(45, 100), (30, 200), (46, 124), (23, 32)]

#mycursor.execute("Drop TABLE Scores")
#mycursor.execute("Drop TABLE Users")


mycursor = db.cursor()

# We can store our commands/queries in str variables
Q1 = "CREATE TABLE Users (id int PRIMARY KEY AUTO_INCREMENT, name varchar(25), username varchar(25), passwd varchar(25))"

# this is how we link the info from one table in the other
Q2 = "CREATE TABLE Scores (userId int PRIMARY KEY, FOREIGN KEY(userId) references Users(id), game1 int DEFAULT 0, game2 int DEFAULT 0)"

mycursor.execute(Q1)
mycursor.execute(Q2)

In [75]:
mycursor.execute("SHOW TABLES")
for x in mycursor:
    print(x)

('scores',)
('users',)


Now lets add the lists "users" and "scores" to the tables

In [76]:
# we can do it like this, adding all listed items at once
#mycursor.executemany("INSERT INTO Users (name, username, passwd) VALUES (%s, %s, %s)", users)

# or we can update both tables with the same for-loop:
Q3 = "INSERT INTO Users (name, username, passwd) VALUES (%s, %s, %s)"
Q4 = "INSERT INTO Scores (userId, game1, game2) VALUES (%s, %s, %s)"

for x, user in enumerate(users):
    mycursor.execute(Q3, user)
    
    last_id = mycursor.lastrowid
    mycursor.execute(Q4, (last_id,) + scores[x])

db.commit()

In [77]:
mycursor.execute("SELECT * FROM Users")
for x in mycursor:
    print(x)

print('\n')
    
mycursor.execute("SELECT * FROM Scores")
for x in mycursor:
    print(x)

(1, 'Doug', 'medoug7', 'senha123')
(2, 'Emilia', 'emil', 'senhaaa')
(3, 'Fred', 'fredfredfred', 'senha1234')
(4, 'Otavia', 'otah', 'senhaaa')


(1, 45, 100)
(2, 30, 200)
(3, 46, 124)
(4, 23, 32)


## test 

In [9]:
# connect to server
db = mysql.connector.connect(host="localhost",
                            user = "root", passwd = "root", database="testdatabase")


trans = [(3213, 20),
         (3213, 30),
         (3213, 50),
         (7624, 10),
         (7624, 5),
         (2121, 25)]

cards = [(4433, 3213), (2134, 7624), (9987, 2121), (8765, 9864)]


mycursor = db.cursor()

mycursor.execute("Drop TABLE Carts")
#mycursor.execute("Drop TABLE Transac")

# We can store our commands/queries in str variables
Q1 = "CREATE TABLE Carts (clientId int, card varchar(10))"
# this is how we link the info from one table in the other
Q2 = "CREATE TABLE Transac (cardId int, FOREIGN KEY(cardId) references Carts(card), value varchar(10))"


mycursor.execute(Q1)
mycursor.execute(Q2)

DatabaseError: 3780 (HY000): Referencing column 'cardId' and referenced column 'card' in foreign key constraint 'transac_ibfk_1' are incompatible.